#### GridSearchCV - 교차 검증과 하이퍼 파라미터 튜닝을 한 번에
+ 하이퍼 파라미터: 머신러닝 알고리즘을 구성하는 주요 구성 요소, 이 값을 조정해 알고리즘의 예측 성능을 개선할 수 있다.
+ 사이킷런은 GridSearchCV API를 이용해 Classifier나 Regressor와 같은 알고리즘에 사용되는 하이퍼 파라미터를 순차적으로 입력하면서 편리하게 최적의 파라미터를 도출할 수 있는 방안을 제공한다.
+ 결정 트리 알고리즘의 여러 하이퍼 파라미터를 순차적으로 변경하면서 최고 성능을 가지는 파라미터 조합을 찾고자 한다면?

In [1]:
grid_parameters = {'max_depth': [1, 2, 3], 'min_sample_split': [2, 3]}

+ 하이퍼 파라미터는 다음과 같이 순차적으로 적용되며, 총 6회에 걸쳐 파라미터를 순차적으로 바꿔 실행하며 최적의 파라미터와 수행 결과를 도출하는 것이 가능하다.

|순번|max_depth|min_sample_split|
|:-:|:-------:|:--------------:|
|1  |1        |2               |
|2  |1        |3               |
|3  |2        |2               |
|4  |2        |3               |
|5  |3        |2               |
|6  |3        |3               |

+ CV가 3회라면, 개별 파라미터 조합마다 3개의 폴딩 세트를 3회에 걸쳐 학습/평가해 성능을 측정
+ 위와 같이 6개의 파라미터 조합이라면, 3회 * 6개 조합 = 18회의 학습 평가가 이루어짐
+ GridSearchCV 클래스의 생성자 파라미터
  - estimator: classifier, regressor, pipeline
  - param_grid: key + list 값을 가지는 딕셔너리
  - scoring: 성능 측정에 사용될 평가 방법 (eg. 'accuracy')
  - cv: 교차 검증을 위해 분할되는 학습/테스트 세트의 개수
  - refit(기본값: True): 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습시킴

In [5]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.2, random_state=121)

dtree = DecisionTreeClassifier()

parameters = {'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]}

In [6]:
import pandas as pd

# param_grid의 하이퍼 파라미터를 3개의 train_test set fold로 나누어 테스트 수행 설정
### refit=True가 기본값임에 유의
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)
grid_dtree.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [7]:
print('GridSearchCV 최적 파라미터: ', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터:  {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도:0.9750


In [9]:
from sklearn.metrics import accuracy_score
estimator = grid_dtree.best_estimator_

pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

테스트 데이터 세트 정확도: 0.9667
